In [1]:
import pyodbc
from datetime import date, datetime, timedelta
import pandas as pd
import win32com.client as win32
from databases import Bawm, Crm,PosicaoDm1,PosicaoDm1Pickle,Boletador
from funcoes_datas import FuncoesDatas
import os
import numpy as np
import xlwings as xw
from dateutil.relativedelta import relativedelta

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
fdt = FuncoesDatas()
crm = Crm()
produtos = crm.rating_produtos()
posicao = PosicaoDm1()
peacke = PosicaoDm1Pickle()
bawm = Bawm()
bol = Boletador()

In [3]:
#retornando dados cadastrais das carteiras
dados_po_cadastro = bawm.po_cadastro(apenas_fundos=False)[['GuidContaCRM','NomeContaCRM']]
dados_crm= crm.account()[['accountid','createdon']].rename(columns={'accountid':'GuidContaCRM'})
dados_crm['GuidContaCRM'] = dados_crm['GuidContaCRM'].str.lower()

In [4]:
#retorna o ultimo dia e penultimo dia
data = fdt.hoje()
ultimo_extrato = data - relativedelta(days=data.day -1)
ultimo_extrato = fdt.workday(ultimo_extrato,-1)
data_fim = datetime.date(ultimo_extrato)

penultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
data_ini = datetime.date(penultimo_extrato)
penultimo_extrato = datetime.date(fdt.workday(penultimo_extrato,-1))
penultimo_extrato = '_' + str(penultimo_extrato.year) + '-' + '{:02d}'.format(penultimo_extrato.month)  + '-' + str(penultimo_extrato.day)

#Transforma a data para ler o peakle
caminho1 = f'O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_fundos{penultimo_extrato}.pickle'
caminho2 = f'O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_tit{penultimo_extrato}.pickle'

# #le o pickle
carteira_fundos = pd.read_pickle(caminho1)
carteira_sc = pd.read_pickle(caminho2)
carteira_penultimo_mes = pd.concat([carteira_fundos,carteira_sc])

#Trazer a carteira do ultimo mes

carteira_ultimo_mes = posicao.posicao_total_depara(ultimo_extrato)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_ultimo_mes=carteira_ultimo_mes.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_ultimo_mes['GuidProduto']= carteira_ultimo_mes['GuidProduto'].str.lower()
#Retirar da carteira Caixa e Compromissada
carteira_ultimo_mes = carteira_ultimo_mes[(carteira_ultimo_mes['TipoProduto']!='CAIXA')&(carteira_ultimo_mes['TipoProduto']!='COMP')]

#Trazer a carteira do penultimo mês

carteira_penultimo_mes = carteira_penultimo_mes[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_penultimo_mes =carteira_penultimo_mes .sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_penultimo_mes ['GuidProduto']= carteira_penultimo_mes ['GuidProduto'].str.lower()
#Retirar da carteira Caixa e Compromissada
carteira_penultimo_mes  = carteira_penultimo_mes [(carteira_penultimo_mes ['TipoProduto']!='CAIXA')&(carteira_penultimo_mes ['TipoProduto']!='COMP')]

#retornar os clientes novos
clientes_ultimo_mes = carteira_ultimo_mes[['NomeContaCRM','TipoProduto']]
clientes_penultimo_mes = carteira_penultimo_mes[['NomeContaCRM','TipoProduto']]
novos = pd.merge(left=clientes_ultimo_mes,right=clientes_penultimo_mes,on='NomeContaCRM',how='left')
clientes_novos = novos[novos['TipoProduto_y'].isnull()].drop_duplicates()
clientes_novos = clientes_novos['NomeContaCRM'].drop_duplicates()

#clientes_novos
clientes_novos = pd.merge(left=clientes_novos,right=dados_po_cadastro,on='NomeContaCRM',how='left')
clientes_novos = pd.merge(left=clientes_novos,right=dados_crm,on='GuidContaCRM',how='left')
clientes_novos = clientes_novos.drop_duplicates().sort_values(by=['createdon'],ascending=False)



In [5]:
#Ativos aprovados por credito
#Produtos aprovados pela casa

pasta = 'O:/SAO/CICH_All/Real Estate/1. Users/Guilherme Liberatore/2. Real Estate/3. Monitoramento/PDFs'
ativos_imob = os.listdir(pasta)
ativos_imob = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_imob]

pasta = 'O:/SAO/CICH_All/Renda Variável/Monitoramento/Monitoramentos/2024.1T'
ativos_rv = os.listdir(pasta)
ativos_rv  = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_rv]

corporativos =  pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')
corporativos  = corporativos[(corporativos['Status']=='Recomendado')]
corporativos['CETIP'] = corporativos['CETIP'].str.lower()
corporativos = list(corporativos['CETIP'])

estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')
estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')[['IdPAS']].astype('str')
estruturados = estruturados[estruturados['IdPAS'].notnull()].dropna()
estruturados = pd.merge(left=estruturados,right=produtos,on='IdPAS',how='left')

estruturados['CETIP'] = estruturados['CETIP'].str.lower()
estruturados['ISIN'] = estruturados['ISIN'].str.lower()
estruturado_cetip = list(estruturados['CETIP'])
estruturado_isin= list(estruturados['ISIN'])

#Gestores aprovados
gestores = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Gestores')[['Gestora']]
gestores =list(gestores['Gestora'].str.lower())

#Códigos dos ativos aprovados

aprovados = ativos_rv + ativos_imob+corporativos+estruturado_cetip+estruturado_isin
aprovados = [aprovado for aprovado in aprovados if str(aprovado) != 'nan']
aprovados = [aprovado for aprovado in aprovados if aprovado != np.nan]
aprovados = [aprovado for aprovado in aprovados if aprovado != None]

produtos_aprovados = pd.DataFrame(aprovados)

In [6]:
#Trazer todos os produtos e suas informações para complementar as decisões do comite

produtos = crm.rating_produtos()
produtos['GuidProduto']=produtos['GuidProduto'].str.lower()
produtos['CETIP']=produtos['CETIP'].str.lower()
produtos['ISIN']=produtos['ISIN'].str.lower()

#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
n_produtos_comite = len(produtos_comite)
produtos_comite_maio = len(produtos_comite[produtos_comite['Data Comitê']>='2024-05-01'])
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Unifica as bases
Ativos_comite = pd.concat([cnpj,isin,cetip])

#Trazer todos os clientes que passaram pelo comite

#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_ultimo_mes['NomeContaCRM'].drop_duplicates().to_list()
substr = Ativos_comite['Cliente'].drop_duplicates().to_list()
#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]
clientes_comite = Filter(string, substr)




In [22]:
ativos_penultimo_mes['GuidProduto'] = ativos_penultimo_mes['GuidProduto'].astype('str')
ativos_ultimo_mes['GuidProduto'] = ativos_ultimo_mes['GuidProduto'].astype('str')

In [23]:
#Traz os ativos novos
deliberacoes = Ativos_comite[['GuidProduto','Data Comitê','Deliberação','Analisado Pelo Comitê']]
ativos_ultimo_mes = carteira_ultimo_mes[['GuidProduto','NomeProduto','TipoProduto']]
ativos_penultimo_mes = carteira_penultimo_mes[['GuidProduto','NomeProduto','TipoProduto']]
ativos_ultimo_mes = ativos_ultimo_mes.reset_index()
ativos_penultimo_mes = ativos_penultimo_mes.reset_index()
ativos_novos = pd.merge(left=ativos_ultimo_mes,right=ativos_penultimo_mes,on='GuidProduto',how='left')
# ativos_novos=ativos_novos[ativos_novos['TipoProduto_y'].isnull()]
# ativos_novos = ativos_novos[['GuidProduto','NomeProduto_x','TipoProduto_x']].rename(columns={'NomeProduto_x':'NomeProduto','TipoProduto_x':'TipoProduto'}).reset_index(drop=True)
# ativos_novos=pd.merge(left=ativos_novos, right=produtos, on='GuidProduto',how='left')
# ativos_novos=pd.merge(left=ativos_novos, right=deliberacoes, on='GuidProduto',how='left')

# #Retirando todos os cod aprovados CRM
# ativos_novos_cetip=list(ativos_novos['CETIP'].str.lower())
# ativos_novos_cetip = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_cetip ]
# ativos_novos['CETIP_verificar'] = ativos_novos_cetip

# # #Retirando todos os cod aprovados CRM
# ativos_novos_ticker=list(ativos_novos['IdPAS'].str.lower())
# ativos_novos_ticker = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_ticker]
# ativos_novos['Ticker_verificar'] = ativos_novos_ticker

# #Bancos emissores aprovados
# emissores_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
# emissores_aprovados = list(emissores_aprovados['Banco'])
# emissores = ativos_novos['emissor']
# emissores_aprovados = ['Recomendado' if emissor in emissores_aprovados else 'Reprovado' for emissor in emissores]
# emissores_aprovados = [emissor for emissor in emissores_aprovados if str(emissor) != 'nan']
# ativos_novos['Emissor_Aprovado'] = emissores_aprovados

MemoryError: Unable to allocate 85.0 MiB for an array with shape (22274680,) and data type int32

In [ ]:
ativos_novos['Recomendado_Investimentos'] = ativos_novos.apply(lambda x: list([x['CETIP_verificar'],x['Ticker_verificar'],x['Emissor_Aprovado']]),axis=1)
comite_list = list(ativos_novos['Recomendado_Investimentos'])
aprovados = []
for x in comite_list:
    if ('Monitorado' in x) or ('Recomendado' in x):
        decisao = 'Recomendado'
    else:
        decisao = 'Reprovado'
    aprovados.append(decisao)   
ativos_novos['Recomendado_Investimentos'] = aprovados
ativos_novos['Rating'].loc[ativos_novos['Recomendado_Investimentos']=='Recomendado'] = 'Recomendado'
ativos_novos = ativos_novos.dropna(subset=['GuidProduto'])

In [ ]:
#Geração de estatistica de ativos (Quantificando o quanto de ativos novos e o seu rating)

ativos_novos = ativos_novos.drop_duplicates(subset=['GuidProduto'])
n_ativos_novos = len(ativos_novos['GuidProduto'].drop_duplicates())
n_ativos_reprovados = len(ativos_novos[(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')])
n_ativos_monitorados = len(ativos_novos[ativos_novos['Rating']=='Monitorado'])

n_ativos_comite = len(ativos_novos[ativos_novos['Deliberação'].notnull()])
n_ativos_comite_reprovados = len(ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')])
n_ativos_comite_monitorados = len(ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']=='Monitorado')])

#Retorna os produtos

ativos_reprovados = ativos_novos[(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')]
ativos_monitorados = ativos_novos[ativos_novos['Rating']=='Monitorado']

ativos_comite = ativos_novos[ativos_novos['Deliberação'].notnull()]
ativos_comite_reprovados = ativos_comite[ativos_comite['Analisado Pelo Comitê']=='Comitê']
ativos_comite_reprovados = ativos_comite_reprovados[ativos_comite_reprovados['Deliberação'].str.contains('Zerar')]
#ativos_comite_reprovados = ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']!='Recomendado')&(ativos_novos['Rating']!='Monitorado')]
ativos_comite_monitorados = ativos_novos[(ativos_novos['Deliberação'].notnull())&(ativos_novos['Rating']=='Monitorado')]

In [ ]:
#Retorna apenas os clientes do comite que entraram na D1

#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_ultimo_mes['NomeContaCRM'].drop_duplicates().to_list()
substr = Ativos_comite['Cliente'].drop_duplicates().to_list()
#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)
def Filter(string, substr):
    return [ str for str in substr if
             any(sub in str for sub in string)]
clientes_entraram = Filter(string, substr)

clientes_entraram = len(clientes_entraram)
clientes_analisados = len(substr)
print(str(clientes_analisados) + ' ' + str(clientes_entraram))

In [ ]:
#Quantos dos clientes novos entraram via comite
def Filter(string, substr):
    return [ str for str in clientes_comite if
             any(sub in str for sub in clientes_novos)]
clientes_novos_comite = Filter(clientes_novos, clientes_comite)

In [ ]:
#Verificar o quanto de ativos que passaram no comite foram vendidos

clientes_comite_sql = ",".join([f"'{item}'" for item in clientes_comite])
posicao_dia= posicao.posicao_range_comite_excecao(data_ini,data_fim,clientes_comite_sql)
boletas_feitas_periodo = boletas = bol.boletas_range_datas(data_ini=data_ini, data_fim=data_fim, exportado=True, deletado=False)
boletas_feitas_periodo = boletas_feitas_periodo[boletas_feitas_periodo['TipoMov']=='V']
boletas_feitas_periodo = boletas_feitas_periodo.rename(columns={'AtivoGuid':'GuidProduto','ContaCRM':'NomeContaCRM'})
boletas_feitas_periodo = boletas_feitas_periodo[['GuidProduto','DataMov','NomeContaCRM','Financeiro',]]
posicao_dia = posicao_dia[['NomeContaCRM','GuidProduto', 'TipoProduto','NomeProduto','FinanceiroFinal']]
posicao_dia = posicao_dia.groupby(['NomeContaCRM','GuidProduto', 'TipoProduto','NomeProduto']).sum()
posicao_dia = posicao_dia.reset_index().drop_duplicates()
posicao_dia['GuidProduto']=posicao_dia['GuidProduto'].str.lower()
ativos_deliberacao = Ativos_comite[['GuidProduto','Data Comitê','Deliberação','Analisado Pelo Comitê']]

#Ativos vendidos esse mês (dos clientes do comitê)
verificar = pd.merge(left=posicao_dia, right=boletas_feitas_periodo, on=['NomeContaCRM','GuidProduto'],how='left')
verificar = pd.merge(left=verificar, right=ativos_deliberacao, on='GuidProduto',how='left')
verificar_vendas = verificar[(verificar['Financeiro'].notnull())&(verificar['Deliberação'].str.contains('Zerar'))]
verificar_vendas = verificar_vendas[verificar_vendas['Analisado Pelo Comitê']=='Comitê']

In [13]:
#Vendas desde o inicio do ano
data_ini = datetime(2023,10,1)
vendas = ativos_deliberacao[ativos_deliberacao['Deliberação'].str.contains('Zerar')][['GuidProduto']].dropna().drop_duplicates()
boletas_feitas_ano = boletas = bol.boletas_range_datas(data_ini=data_ini, data_fim=data_fim, exportado=True, deletado=False)
boletas_feitas_ano = boletas_feitas_ano.reset_index()
boletas_feitas_ano = boletas_feitas_ano.loc[boletas_feitas_ano['ContaCRM'].isin(clientes_comite)]
boletas_feitas_ano = boletas_feitas_ano[boletas_feitas_ano['TipoMov']=='V']
vendas = list(vendas['GuidProduto'])
boletas_feitas_ano = boletas_feitas_ano.loc[boletas_feitas_ano['AtivoGuid'].isin(vendas)]
boletas_feitas_ano.to_excel('vendas_2023.xlsx')




ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
#Verificar quantos clinetes do comitê entraram na carteira

carteira_clientes_comite = posicao.posicao_total_depara(fdt.hoje())
#clientes_comite_d1 = list(clientes_carteira['NomeContaCRM'].drop_duplicates())

#Quantos dos clientes novos entraram via comite
def Filter(string, substr):
    return [ str for str in substr if
             any(sub in str for sub in clientes_comite)]
clientes_d1_comite = Filter(clientes_comite, substr)
print(f'Dos {str(len(substr))} analisados via comitê de exceção, {str(len(clientes_d1_comite))} entraram para nossa gestão ')


In [ ]:
#Verificar ativos do comitê que estão na carteira
carteira_clientes_comite = carteira_clientes_comite.loc[carteira_clientes_comite['NomeContaCRM'].isin(clientes_comite)][['NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']].drop_duplicates().sort_values(by='NomeContaCRM').reset_index(drop=True)
carteira_clientes_comite = pd.merge(left = carteira_clientes_comite, right=produtos, on='GuidProduto',how='left')

In [ ]:
#Retirando todos os cod aprovados CRM
ativos_novos_cetip=list(carteira_clientes_comite['CETIP'].str.lower())
ativos_novos_cetip = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_cetip ]
carteira_clientes_comite['CETIP_verificar'] = ativos_novos_cetip

# #Retirando todos os cod aprovados CRM
ativos_novos_ticker=list(carteira_clientes_comite['IdPAS'].str.lower())
ativos_novos_ticker = ['Recomendado' if (ativo in aprovados) else np.nan for ativo in ativos_novos_ticker]
carteira_clientes_comite['Ticker_verificar'] = ativos_novos_ticker

#Bancos emissores aprovados
emissores_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
emissores_aprovados = list(emissores_aprovados['Banco'])
emissores = carteira_clientes_comite['emissor']
emissores_aprovados = ['Recomendado' if emissor in emissores_aprovados else 'Reprovado' for emissor in emissores]
emissores_aprovados = [emissor for emissor in emissores_aprovados if str(emissor) != 'nan']
carteira_clientes_comite['Emissor_Aprovado'] = emissores_aprovados

carteira_clientes_comite['Recomendado_Investimentos'] = carteira_clientes_comite.apply(lambda x: list([x['CETIP_verificar'],x['Ticker_verificar'],x['Emissor_Aprovado']]),axis=1)
comite_list = list(carteira_clientes_comite['Recomendado_Investimentos'])
aprovados = []
for x in comite_list:
    if ('Monitorado' in x) or ('Recomendado' in x):
        decisao = 'Recomendado'
    else:
        decisao = 'Reprovado'
    aprovados.append(decisao)   
carteira_clientes_comite['Recomendado_Investimentos'] = aprovados
carteira_clientes_comite['Rating'].loc[ativos_novos['Recomendado_Investimentos']=='Recomendado'] = 'Recomendado'
ativos_novos = ativos_novos.dropna(subset=['GuidProduto'])